In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from matplotlib.colors import ListedColormap
import plotly.graph_objs as go

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import joblib

In [2]:
def plot_results(xlabel, df, predictions, sonic, v_label, r_label, vrh_label, xlim_lr, xlim_upr, ylim_lr = False, ylim_upr = False, bckg = False):
    """
    Plots various geological log data and machine learning predictions as a function of depth.
    
    Parameters:
    - xlabel (str): Label for the x-axis.
    - df (DataFrame): Pandas DataFrame containing the well log data.
    - predictions (array-like): Predicted values from a machine learning model.
    - sonic (array-like): Sonic log values from the DataFrame.
    - v_label (str): Column name in df for Voigt boundary data.
    - r_label (str): Column name in df for Reuss boundary data.
    - vrh_label (str): Column name in df for Voigt-Reuss-Hill average data.
    - xlim_lr (float): Lower limit for the x-axis range.
    - xlim_upr (float): Upper limit for the x-axis range.
    - ylim_lr (float, optional): Lower limit for the y-axis range. Defaults to False, which auto-scales.
    - ylim_upr (float, optional): Upper limit for the y-axis range. Defaults to False, which auto-scales.
    - bckg (str or False, optional): Background color for the area between Voigt and Reuss boundaries. If False, yellow is used as the default color.
    
    The function creates a vertical plot typically used for well log representation with depth increasing downwards. 
    It plots the sonic log, machine learning predictions, and the Voigt-Reuss-Hill average as lines on the same plot. 
    It also shades the area between the Voigt and Reuss boundaries to indicate uncertainty or variability range. 
    The function allows for customization of the x and y-axis limits and the background color for the shaded area.
    """

    depth = df['DEPT']
    voigt_boundary = df[v_label]
    reuss_boundary = df[r_label]
    vp_vrh = df[vrh_label]
    xlen = df.shape[0]
    
    plt.figure(figsize=(9, 10))
    plt.gca().invert_yaxis()
    plt.gca().xaxis.tick_top()
    plt.gca().xaxis.set_label_position('top')
    
    for _, el in formation_depths.iterrows():
        plt.axhline(y=el['Min Value'], color='grey', linestyle='--', linewidth=1, alpha=0.3)
        plt.axhline(y=el['Max Value'], color='grey', linestyle='--', linewidth=1, alpha=0.3)
        
#         if np.isfinite(el['Min Value']) and np.isfinite(el['Max Value']):
#               plt.fill_betweenx(y=[el['Min Value'], el['Max Value']], 
#                                 x1=xlim_lr, x2=xlim_upr,
#                                 color=el['color'], alpha=0.1)
                
    plt.plot(sonic, depth, label='Sonic Log', alpha=1)
    plt.plot(predictions, depth, label='ML Predictions', alpha=0.8, color='red')
    plt.plot(vp_vrh, depth, label='Voigt Reuss Hill', alpha=0.5, color='black')
    plt.plot(voigt_boundary, depth,  alpha=0.8, linewidth=0.4, color='violet')
    plt.plot(reuss_boundary, depth, label='Voigt-Reuss Boundaries', alpha=0.8, linewidth=0.4, color='violet')
    
    if bckg:
        plt.fill_betweenx(depth, voigt_boundary, reuss_boundary, where=(voigt_boundary >= reuss_boundary), 
                      interpolate=True, color=bckg, alpha=0.5)
    else:
        plt.fill_betweenx(depth, voigt_boundary, reuss_boundary, where=(voigt_boundary >= reuss_boundary), 
                      interpolate=True, color='yellow', alpha=0.5)
    
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), shadow=True, ncol=2, 
               framealpha=1.0, edgecolor='black', facecolor='white', frameon=True)
    
    plt.ylabel('Depth [m]', fontweight='bold', labelpad=15)
    plt.xlabel(xlabel, fontweight='bold', labelpad=15)
    
    plt.rcParams['font.size'] = 20
    plt.rcParams['font.family'] = 'arial'
    
    plt.xlim(xlim_lr, xlim_upr)
    if (ylim_lr and ylim_upr):
        plt.ylim(ylim_lr, ylim_upr)

    plt.show()

In [3]:
def plot_true_vs_predicted(ax, y_true, y_pred, title):
    """
    Plots true vs predicted values with a reference line (y=x).
    
    Parameters:
    - ax: Matplotlib Axes object to plot on.
    - y_true: Array-like, true target values.
    - y_pred: Array-like, predicted values.
    - title: String, title of the plot.
    """
    ax.scatter(y_true, y_pred, alpha=0.5)
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
    ax.set_title(title)
    ax.set_xlabel('True Values')
    ax.set_ylabel('Predictions')

def plot_true_vs_predicted_v1(ax, true_data, predicted_data, label, marker, color):
    """
    Enhanced true vs predicted plot with custom styling and legend.
    
    Parameters:
    - ax: Matplotlib Axes object.
    - true_data: Array-like, true values.
    - predicted_data: Array-like, predicted values.
    - label: String, legend label.
    - marker: String, marker style.
    - color: String, color of the plot points.
    """
    ax.scatter(true_data, predicted_data, alpha=0.7, label=label, marker=marker, color=color)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    ax.legend()

def plot_true_vs_predicted_v2(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    """
    True vs predicted plot with MAE and R² metrics displayed.
    
    Parameters:
    - ax: Matplotlib Axes object.
    - true_data: Array-like, true values.
    - predicted_data: Array-like, predicted values.
    - label: String, legend label.
    - marker: String, marker style.
    - color: String, color of the plot points.
    - x_pos, y_pos: Floats, position of metrics text on the plot (relative axes coordinates).
    """
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.5, label=f'{label}', color=color, marker=marker)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', 
            transform=ax.transAxes, verticalalignment='top', horizontalalignment='left', fontsize=10, 
            bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

def plot_true_vs_vrh(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    """
    True vs VRH values plot with MAE and R² metrics displayed.
    
    Parameters:
    - ax: Matplotlib Axes object.
    - true_data: Array-like, true values.
    - predicted_data: Array-like, VRH values.
    - label: String, legend label.
    - marker: String, marker style.
    - color: String, color of the plot points.
    - x_pos, y_pos: Floats, position of metrics text on the plot (relative axes coordinates).
    """
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.4, label=f'{label}', color=color, marker=marker)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('VRH Values', fontweight='bold')
    ax.set_title('True vs VRH', fontweight='bold')
    plt.xlim(2000, 7000)
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', 
            transform=ax.transAxes, verticalalignment='top', horizontalalignment='left', 
            fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

In [4]:
def calculate_voigt_reuss_bounds(df, minerals, bulk_modulus, shear_modulus):
    """
    Calculates the Voigt and Reuss bounds for bulk and shear moduli based on mineralogical composition.

    This function computes the Voigt and Reuss bounds for bulk and shear moduli 
    for a given dataset of mineral volume fractions. It also calculates the average 
    moduli (Voigt-Reuss-Hill averages) and adds the results as new columns to the input DataFrame.

    Parameters:
    -----------
    df (pandas.DataFrame): 
        A DataFrame containing the volume fractions of minerals for each sample.
    minerals (list): 
        A list of column names representing the minerals in the DataFrame.
    bulk_modulus (dict): 
        A dictionary mapping each mineral to its bulk modulus (GPa).
    shear_modulus (dict): 
        A dictionary mapping each mineral to its shear modulus (GPa).

    Returns:
    --------
    pandas.DataFrame:
        The input DataFrame with the following additional columns:
        - `Voigt_Bulk`: Voigt bound for bulk modulus.
        - `Reuss_Bulk`: Reuss bound for bulk modulus.
        - `Voigt_Shear`: Voigt bound for shear modulus.
        - `Reuss_Shear`: Reuss bound for shear modulus.
        - `K_VRH`: Average bulk modulus (Voigt-Reuss-Hill average).
        - `G_VRH`: Average shear modulus (Voigt-Reuss-Hill average).

    Calculation Details:
    --------------------
    - Voigt Bound: Assumes a uniform strain distribution and is calculated as the weighted sum 
      of the mineral moduli (bulk or shear) using their volume fractions.
    - Reuss Bound: Assumes a uniform stress distribution and is calculated as the reciprocal of 
      the weighted sum of the reciprocal mineral moduli (bulk or shear) using their volume fractions.
    - Voigt-Reuss-Hill Average (VRH): The average of the Voigt and Reuss bounds.

    Example:
    --------
    Given the following input:
    - `df` with columns `Mineral1`, `Mineral2`, ... containing volume fractions.
    - `bulk_modulus = {'Mineral1': 40, 'Mineral2': 60}`.
    - `shear_modulus = {'Mineral1': 20, 'Mineral2': 30}`.

    The function computes the Voigt, Reuss, and VRH averages for each row in the DataFrame.

    Usage:
    ------
    df = calculate_voigt_reuss_bounds(df, minerals, bulk_modulus, shear_modulus)
    """
    dataset = df[minerals]
    voigt_bulk = []
    reuss_bulk = []
    voigt_shear = []
    reuss_shear = []
    
    for index, row in dataset.iterrows():
        sum_voigt_bulk = sum_voigt_shear = 0
        sum_reuss_bulk = sum_reuss_shear = 0
        for mineral in minerals:
            volume_fraction = row[mineral]
            sum_voigt_bulk += volume_fraction * bulk_modulus[mineral]
            sum_voigt_shear += volume_fraction * shear_modulus[mineral]
            # Avoid division by zero
            if bulk_modulus[mineral] > 0:
                sum_reuss_bulk += volume_fraction / bulk_modulus[mineral]
            if shear_modulus[mineral] > 0:
                sum_reuss_shear += volume_fraction / shear_modulus[mineral]
        voigt_bulk.append(sum_voigt_bulk)
        voigt_shear.append(sum_voigt_shear)
        reuss_bulk_value = 1 / sum_reuss_bulk if sum_reuss_bulk > 0 else 0
        reuss_shear_value = 1 / sum_reuss_shear if sum_reuss_shear > 0 else 0
        reuss_bulk.append(reuss_bulk_value)
        reuss_shear.append(reuss_shear_value)
        
        df.loc[index, 'Voigt_Bulk'] = sum_voigt_bulk
        df.loc[index, 'Reuss_Bulk'] = reuss_bulk_value
        df.loc[index, 'Voigt_Shear'] = sum_voigt_shear
        df.loc[index, 'Reuss_Shear'] = reuss_shear_value
    
    df['K_VRH'] = (df['Voigt_Bulk'] + df['Reuss_Bulk']) / 2
    df['G_VRH'] = (df['Voigt_Shear'] + df['Reuss_Shear']) / 2
    
    return df

In [5]:
# Importing the Data
df = pd.read_csv("Datasets/Training and Testing data/df.csv")

train_df_vp = pd.read_csv("DATASETS/Training and Testing data/train_df_vp.csv")
test_df_vp = pd.read_csv("DATASETS/Training and Testing data/test_df_vp.csv")
test_df2_vp = pd.read_csv("DATASETS/Training and Testing data/test_df2_vp.csv")

train_df_vs = pd.read_csv("DATASETS/Training and Testing data/train_df_vs.csv")
test_df_vs = pd.read_csv("DATASETS/Training and Testing data/test_df_vs.csv")
test_df2_vs = pd.read_csv("DATASETS/Training and Testing data/test_df2_vs.csv")

formations = pd.read_excel("DATASETS/Input data/Rock Formation Depths.xlsx")

In [6]:
# Generating a Dataframe with required rock formation Depths

layers = {
    "Layer": [
        "Eau Claire A-C", "Mt. Simon E", "Mt. Simon D", "Mt. Simon C",
        "Mt. Simon B", "Mt. Simon A", "Argenta", "Precambrian rhyolite"
    ],
    "Min Value": [5380, 5552, 5907, 6117, 6428, 6668, 7043, 7162],
    "Max Value": [5552, 5907, 6117, 6428, 6668, 7043, 7162, 8000],
    "color": ['grey','purple','purple','purple','purple','purple','red','blue']
}
formation_depths = pd.DataFrame(layers)

In [7]:
formation_depths.head(10)

,Layer,Min Value,Max Value,color
0,Eau Claire A-C,5380,5552,grey
1,Mt. Simon E,5552,5907,purple
2,Mt. Simon D,5907,6117,purple
3,Mt. Simon C,6117,6428,purple
4,Mt. Simon B,6428,6668,purple
5,Mt. Simon A,6668,7043,purple
6,Argenta,7043,7162,red
7,Precambrian rhyolite,7162,8000,blue


In [8]:
def feet_to_meters(feet):
    return feet * 0.3048

test_df_vs['DEPT'] = test_df_vs['DEPT'].apply(feet_to_meters)
test_df_vp['DEPT'] = test_df_vp['DEPT'].apply(feet_to_meters)

test_df2_vs['DEPT'] = test_df2_vs['DEPT'].apply(feet_to_meters)
test_df2_vp['DEPT'] = test_df2_vp['DEPT'].apply(feet_to_meters)

In [9]:
formation_depths['Min Value'] = formation_depths['Min Value'].apply(feet_to_meters)
formation_depths['Max Value'] = formation_depths['Max Value'].apply(feet_to_meters)

In [10]:
formation_depths.head(10)

,Layer,Min Value,Max Value,color
0,Eau Claire A-C,1639.8240,1692.2496,grey
1,Mt. Simon E,1692.2496,1800.4536,purple
2,Mt. Simon D,1800.4536,1864.4616,purple
3,Mt. Simon C,1864.4616,1959.2544,purple
4,Mt. Simon B,1959.2544,2032.4064,purple
5,Mt. Simon A,2032.4064,2146.7064,purple
6,Argenta,2146.7064,2182.9776,red
7,Precambrian rhyolite,2182.9776,2438.4000,blue


# Random Forest

## TRAINING FOR VP

In [11]:
target_column = 'VP'
predictor_cols = ['BOUND_WATER','CALCITE','CHLORITE','DOLOMITE',#'HEMATITE',
                    'ILLITE','K-FELDSPAR','KAOLINITE','QUARTZ','UWATER']

X = train_df_vp[predictor_cols]
y = train_df_vp[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [12]:
# Define a scoring function
scoring = {
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'R2': 'r2'
}

param_grid = {
    'n_estimators': [20],  # Number of trees in the forest
    'min_samples_split': [5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [2, 4]  # Minimum number of samples required at each leaf node
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring=scoring, refit='R2', return_train_score=True)
grid_search.fit(X_train_scaled, y_train)
best_rf = grid_search.best_estimator_
print('Best parameters found: ', grid_search.best_params_)

val_predictions = best_rf.predict(X_val_scaled)
val_mae = mean_absolute_error(y_val, val_predictions)
val_r2 = r2_score(y_val, val_predictions)
print('Validation MAE: ', val_mae)
print('Validation R-squared: ', val_r2)

Best parameters found:  {'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 20}
Validation MAE:  155.82420061063885
Validation R-squared:  0.945055610646722


In [13]:
filename = './../Model Weights/Random Forest for P-Wave Velocity/random_forest_vp.joblib'
joblib.dump(best_rf, filename, compress=3)

FileNotFoundError: [Errno 2] No such file or directory: './../Model Weights/Random Forest for P-Wave Velocity/random_forest_vp.joblib'

In [ ]:
filename = './../Model Weights/Random Forest for P-Wave Velocity/random_forest_vp.joblib'
best_rf = joblib.load(filename)

In [ ]:
# Evaluate the best model on the validation set
val_predictions = best_rf.predict(X_val_scaled)
val_mae = mean_absolute_error(y_val, val_predictions)
val_r2 = r2_score(y_val, val_predictions)

print('Validation MAE: ', val_mae)
print('Validation R-squared: ', val_r2)

In [ ]:
X_test = test_df_vp.drop(['VP', 'RHOZ', 'VOIGT_VP', 'REUSS_VP', 'VRH_VP', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df_vp['VP']
X_test_scaled = scaler.transform(X_test)

y_test_pred_vp = best_rf.predict(X_test_scaled)
y_val_pred = best_rf.predict(X_val_scaled)
y_train_pred = best_rf.predict(X_train_scaled)

train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

test_rmse = mean_squared_error(y_test, y_test_pred_vp, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_vp)
test_r2 = r2_score(y_test, y_test_pred_vp)

vrh_rmse = mean_squared_error(y_test, list(test_df_vp['VRH_VP']), squared=False)
vrh_mae = mean_absolute_error(y_test, list(test_df_vp['VRH_VP']))
vrh_r2 = r2_score(y_test, list(test_df_vp['VRH_VP']))

### 2. Plotting predictions with True Values

In [ ]:
plt.figure(figsize=(21, 7))

# Plot for the training set
ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted(ax1, y_train, y_train_pred, 'Training Set - True vs Predicted')

# Plot for the validation set
ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted(ax2, y_val, y_val_pred, 'Validation Set - True vs Predicted')

# Plot for the test set
ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted(ax3, y_test, y_test_pred_vp, 'Test Set - True vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(1, 1, 1)

plot_true_vs_predicted_v1(ax, y_test, y_test_pred_vp, 'Test Set', 'o', 'red')
plot_true_vs_predicted_v1(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue')
plot_true_vs_predicted_v1(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)

plot_true_vs_predicted_v2(ax, y_test, y_test_pred_vp, 'Testing Set', 'o', 'red', 0.66, -0.17)
plot_true_vs_predicted_v2(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue', 0.4, -0.17)
plot_true_vs_predicted_v2(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow', 0.15, -0.17)

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

In [ ]:
def calculate_vpvrh(df):
    # (1 GPa = 1e9 Pa)
    KVRH = df['K_VRH'] * 1e9
    GVRH = df['G_VRH'] * 1e9
    
    # (1 g/cm^3 = 1000 kg/m^3)
    rho = df['RHOZ'] * 1000
    
    df['VRH_VP'] = np.sqrt((KVRH + 4.0/3.0 * GVRH) / rho)
    return df

train_df_vp = calculate_vpvrh(train_df_vp)

In [ ]:
def plot_true_vs_vrh(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.4, label=f'{label}', color=color, marker=marker)
    ax.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('VRH Values', fontweight='bold')
    ax.set_title('True vs VRH', fontweight='bold')
    plt.xlim(1500, 7200)
    plt.ylim(1500, 7200)
    # Set text for metrics below the plot area
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', transform=ax.transAxes, verticalalignment='top', horizontalalignment='left', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)

plot_true_vs_vrh(ax, df['VP'], df['VRH_VP'], 'VRH', 'o', 'pink', 0.66, -0.17)

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)

plot_true_vs_predicted_v2(ax, y_test, y_test_pred_vp, 'Testing Set', 'o', 'red', 0.66, -0.17)
plot_true_vs_vrh(ax, y_test, test_df_vp['VRH_VP'], 'VRH', 'o', 'pink', 0.66, -0.17)

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

### 3. Plotting Results

In [ ]:
plot_results('P-Wave Velocity [m/s]', test_df_vp, y_test_pred_vp, y_test, 'VOIGT_VP', 'REUSS_VP','VRH_VP', 0, 7500, 2200, 1550)

## TRAINING FOR VS

In [ ]:
target_column = 'VS'
predictor_cols = ['BOUND_WATER','CALCITE','CHLORITE','DOLOMITE',
                  'ILLITE','K-FELDSPAR','KAOLINITE','QUARTZ','UWATER']

X = train_df_vs[predictor_cols]
y = train_df_vs[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=4)
model.fit(X_train_scaled, y_train)
joblib.dump(model, './../Model Weights/Random Forest for S-Wave Velocity/random_forest_vs.joblib', compress=3)

train_predictions = model.predict(X_train_scaled)
val_predictions = model.predict(X_val_scaled)

X_test = test_df_vs.drop(['VS', 'RHOZ', 'VOIGT_VS', 'REUSS_VS', 'VRH_VS', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df_vs['VS']
X_test_scaled = scaler.transform(X_test)

test_predictions = model.predict(X_test_scaled)

plt.figure(figsize=(21, 7))

ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted(ax1, y_train, train_predictions, 'Training Set - True vs Predicted')

ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted(ax2, y_val, val_predictions, 'Validation Set - True vs Predicted')

ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted(ax3, y_test, test_predictions, 'Test Set - True vs Predicted')

plt.tight_layout()
plt.show()

train_rmse = mean_squared_error(y_train, train_predictions, squared=False)
train_mae = mean_absolute_error(y_train, train_predictions)
train_r2 = r2_score(y_train, train_predictions)

val_rmse = mean_squared_error(y_val, val_predictions, squared=False)
val_mae = mean_absolute_error(y_val, val_predictions)
val_r2 = r2_score(y_val, val_predictions)

test_rmse = mean_squared_error(y_test, test_predictions, squared=False)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

vrh_rmse = mean_squared_error(y_test, list(test_df_vs['VRH_VS']), squared=False)
vrh_mae = mean_absolute_error(y_test, list(test_df_vs['VRH_VS']))
vrh_r2 = r2_score(y_test, list(test_df_vs['VRH_VS']))

# Neural Network

## TRAINING FOR VP

### 1. Building and Training the model

In [ ]:
target_column = 'VP'
predictor_cols = ['BOUND_WATER','CALCITE','CHLORITE','DOLOMITE',#'HEMATITE',
                    'ILLITE','K-FELDSPAR','KAOLINITE','QUARTZ','UWATER']

X = train_df_vp[predictor_cols]
y = train_df_vp[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
# Defining the Neural Network with added complexity
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(len(predictor_cols), 256)  
        self.fc2 = nn.Linear(256, 64) 
        self.fc4 = nn.Linear(64, 1)  
        self.relu = nn.ReLU()  # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc4(x)
        return x

model = Net()

X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32))
X_val_tensor = torch.tensor(X_val_scaled.astype(np.float32))
y_val_tensor = torch.tensor(y_val.values.astype(np.float32))

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.view(-1, 1))
val_dataset = TensorDataset(X_val_tensor, y_val_tensor.view(-1, 1))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Criterion for loss (still needed for the gradient computation)
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    total_mae = 0
    total_r2 = 0
    count = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        # Computing MAE and R-squared for the current batch
        mae = mean_absolute_error(targets.detach().numpy(), outputs.detach().numpy())
        r2 = r2_score(targets.detach().numpy(), outputs.detach().numpy())
        total_mae += mae
        total_r2 += r2
        count += 1
    
    # Computing the average MAE and R-squared over all batches
    avg_mae = total_mae / count
    avg_r2 = total_r2 / count
    if((epoch+1)%10==0 or epoch==0):
        print(f'Epoch {epoch+1}, MAE: {avg_mae:.4f}, R-squared: {avg_r2:.4f}')

In [ ]:
# Save the model weights
joblib.dump(model, './../Model weights/Neural Network for P-Wave Velocity/nn_vp.joblib')
print("Model weights saved as 'nn_vp.joblib'")

In [ ]:
filename = './../Model weights/Neural Network for P-Wave Velocity/nn_vp.joblib'
model = joblib.load(filename)

In [ ]:
# TEST SET PREDICTIONS
X_test = test_df_vp.drop(['VP', 'RHOZ', 'VOIGT_VP', 'REUSS_VP', 'VRH_VP', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df_vp['VP']
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Making Predictions
X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32))
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_predictions_np = test_predictions.numpy().flatten()
    print("Predictions:", test_predictions_np)
    
model.eval()
with torch.no_grad():
    train_predictions = model(X_train_tensor)
    train_predictions_np = train_predictions.numpy().flatten()
    print("Predictions:", train_predictions_np)
    
model.eval()
with torch.no_grad():
    val_predictions = model(X_val_tensor)
    val_predictions_np = val_predictions.numpy().flatten()
    print("Predictions:", val_predictions_np)

### 2. Plotting predictions with True Values

In [ ]:
plt.figure(figsize=(21, 7))

# Plot for the training set
ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted(ax1, y_train, train_predictions_np, 'Training Set - True vs Predicted')

# Plot for the validation set
ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted(ax2, y_val, val_predictions_np, 'Validation Set - True vs Predicted')

# Plot for the test set
ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted(ax3, y_test, test_predictions_np, 'Test Set - True vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
def plot_true_vs_predicted_v1(ax, true_data, predicted_data, label, marker, color):
    ax.scatter(true_data, predicted_data, alpha=0.7, label=label, marker=marker, color=color)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    ax.legend()

plt.figure(figsize=(10, 10))
ax = plt.subplot(1, 1, 1)

plot_true_vs_predicted_v1(ax, y_test, test_predictions_np, 'Test Set', 'o', 'red')
plot_true_vs_predicted_v1(ax, y_val, val_predictions_np, 'Validation Set', 's', 'blue')
plot_true_vs_predicted_v1(ax, y_train, train_predictions_np, 'Training Set', '^', 'yellow')

plt.tight_layout()
plt.show()

In [ ]:
def plot_true_vs_predicted_v2(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.5, label=f'{label}', color=color, marker=marker)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    # Set text for metrics below the plot area
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', transform=ax.transAxes, verticalalignment='top', horizontalalignment='left', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)

# Adjust x_pos and y_pos for each dataset to display metrics in one horizontal line below the graph
plot_true_vs_predicted_v2(ax, y_test, test_predictions_np, 'Testing Set', 'o', 'red', 0.66, -0.17)
plot_true_vs_predicted_v2(ax, y_val, val_predictions_np, 'Validation Set', 's', 'blue', 0.4, -0.17)
plot_true_vs_predicted_v2(ax, y_train, train_predictions_np, 'Training Set', '^', 'yellow', 0.15, -0.17)

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

### 3. Comparing Metrics

In [ ]:
# Calculate error metrics for the training set
train_rmse = mean_squared_error(y_train, train_predictions_np, squared=False)
train_mae = mean_absolute_error(y_train, train_predictions_np)
train_r2 = r2_score(y_train, train_predictions_np)

# Calculate error metrics for the validation set
val_rmse = mean_squared_error(y_val, val_predictions_np, squared=False)
val_mae = mean_absolute_error(y_val, val_predictions_np)
val_r2 = r2_score(y_val, val_predictions_np)

# Calculate error metrics for the Test set
test_rmse = mean_squared_error(y_test, test_predictions_np, squared=False)
test_mae = mean_absolute_error(y_test, test_predictions_np)
test_r2 = r2_score(y_test, test_predictions_np)

# Calculate metrics for the VRH values
vrh_rmse = mean_squared_error(y_test, list(test_df_vp['VRH_VP']), squared=False)
vrh_mae = mean_absolute_error(y_test, list(test_df_vp['VRH_VP']))
vrh_r2 = r2_score(y_test, list(test_df_vp['VRH_VP']))

### 4. Plotting Results

In [ ]:
plot_results('P-Wave Velocity [m/s]', test_df_vp, test_predictions_np, y_test, 'VOIGT_VP', 'REUSS_VP','VRH_VP', 0, 7500, 2200, 1550)

## TRAINING FOR VS

In [ ]:
target_column = 'VS'
predictor_cols = ['BOUND_WATER','CALCITE','CHLORITE','DOLOMITE',#'HEMATITE',
                    'ILLITE','K-FELDSPAR','KAOLINITE','QUARTZ','UWATER']

X = train_df_vs[predictor_cols]
y = train_df_vs[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
# Defining the Neural Network with added complexity
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(len(predictor_cols), 256)  
        self.fc2 = nn.Linear(256, 64) 
        self.fc4 = nn.Linear(64, 1)  
        self.relu = nn.ReLU()  # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc4(x)
        return x

model = Net()

# Assume X_train_scaled, y_train, X_val_scaled, y_val are defined elsewhere
# Prepare Data for PyTorch
X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32))
X_val_tensor = torch.tensor(X_val_scaled.astype(np.float32))
y_val_tensor = torch.tensor(y_val.values.astype(np.float32))

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.view(-1, 1))
val_dataset = TensorDataset(X_val_tensor, y_val_tensor.view(-1, 1))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Criterion for loss (still needed for the gradient computation)
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  

num_epochs = 500

model.train()

for epoch in range(num_epochs):
    total_mae = 0
    total_r2 = 0
    count = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        # Computing MAE and R-squared for the current batch
        mae = mean_absolute_error(targets.detach().numpy(), outputs.detach().numpy())
        r2 = r2_score(targets.detach().numpy(), outputs.detach().numpy())
        total_mae += mae
        total_r2 += r2
        count += 1
    
    # Computing the average MAE and R-squared over all batches
    avg_mae = total_mae / count
    avg_r2 = total_r2 / count
    if((epoch+1)%10==0 or epoch==0):
        print(f'Epoch {epoch+1}, MAE: {avg_mae:.4f}, R-squared: {avg_r2:.4f}')

In [ ]:
joblib.dump(model, './../Model weights/Neural Network for S-Wave Velocity/nn_vs.joblib')
print("Model weights saved as 'nn_vs.joblib'")

In [ ]:
filename = './../Model weights/Neural Network for S-Wave Velocity/nn_vs.joblib'
model = joblib.load(filename)

In [ ]:
# TEST SET PREDICTIONS
X_test = test_df_vs.drop(['VS', 'RHOZ', 'VOIGT_VS', 'REUSS_VS', 'VRH_VS', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df_vs['VS']
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Making Predictions
X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32))
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_predictions_np = test_predictions.numpy().flatten()
    print("Predictions:", test_predictions_np)
    
model.eval()
with torch.no_grad():
    train_predictions = model(X_train_tensor)
    train_predictions_np = train_predictions.numpy().flatten()
    print("Predictions:", train_predictions_np)
    
model.eval()
with torch.no_grad():
    val_predictions = model(X_val_tensor)
    val_predictions_np = val_predictions.numpy().flatten()
    print("Predictions:", val_predictions_np)

In [ ]:
plt.figure(figsize=(21, 7))

ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted(ax1, y_train, train_predictions_np, 'Training Set - True vs Predicted')

ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted(ax2, y_val, val_predictions_np, 'Validation Set - True vs Predicted')

ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted(ax3, y_test, test_predictions_np, 'Test Set - True vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate error metrics for the training set
train_rmse = mean_squared_error(y_train, train_predictions_np, squared=False)
train_mae = mean_absolute_error(y_train, train_predictions_np)
train_r2 = r2_score(y_train, train_predictions_np)

# Calculate error metrics for the validation set
val_rmse = mean_squared_error(y_val, val_predictions_np, squared=False)
val_mae = mean_absolute_error(y_val, val_predictions_np)
val_r2 = r2_score(y_val, val_predictions_np)

# Calculate error metrics for the Test set
test_rmse = mean_squared_error(y_test, test_predictions_np, squared=False)
test_mae = mean_absolute_error(y_test, test_predictions_np)
test_r2 = r2_score(y_test, test_predictions_np)

# Calculate metrics for the VRH values
vrh_rmse = mean_squared_error(y_test, list(test_df_vs['VRH_VS']), squared=False)
vrh_mae = mean_absolute_error(y_test, list(test_df_vs['VRH_VS']))
vrh_r2 = r2_score(y_test, list(test_df_vs['VRH_VS']))

## XGBoost

### 1. Training for Vp 

In [ ]:
train_df_vp.head()

In [ ]:
target_column = 'VP'
predictor_cols = ['BOUND_WATER','CALCITE','CHLORITE','DOLOMITE',#'HEMATITE',
                    'ILLITE','K-FELDSPAR','KAOLINITE','QUARTZ','UWATER']

X = train_df_vp[predictor_cols]
y = train_df_vp[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [50, 100, 200],
    'subsample': [0.7, 1],
    'colsample_bytree': [0.7, 1],
    'reg_alpha': [ 0.1]
}

xgb_model = XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train_scaled, y_train)

print(f"Best parameters found: {grid_search.best_params_}")

best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val_scaled)

In [ ]:
filename = 'Model weights/XGBoost for P-Wave Velocity/best_xgb_model.joblib'
joblib.dump(best_model, filename)

In [ ]:
filename = 'Model weights/XGBoost for P-Wave Velocity/best_xgb_model.joblib'
best_model = joblib.load(filename)

In [ ]:
y_val_pred = best_model.predict(X_val_scaled)
y_train_pred = best_model.predict(X_train_scaled)

In [ ]:
best_model

In [ ]:
# TEST SET PREDICTIONS
X_test = test_df_vp.drop(['VP', 'RHOZ', 'VOIGT_VP', 'REUSS_VP', 'VRH_VP', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df_vp['VP']
X_test_scaled = scaler.transform(X_test)

In [ ]:
y_test_pred_vp = best_model.predict(X_test_scaled)

In [ ]:
# Calculate error metrics for the training set
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate error metrics for the validation set
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Calculate error metrics for the test set
test_rmse = mean_squared_error(y_test, y_test_pred_vp, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_vp)
test_r2 = r2_score(y_test, y_test_pred_vp)

# Calculate metrics for the VRH values
vrh_rmse = mean_squared_error(y_test, list(test_df_vp['VRH_VP']), squared=False)
vrh_mae = mean_absolute_error(y_test, list(test_df_vp['VRH_VP']))
vrh_r2 = r2_score(y_test, list(test_df_vp['VRH_VP']))

fig = create_metrics_table_plotly(train_rmse, train_mae, train_r2, val_rmse, val_mae, val_r2, test_rmse, test_mae, test_r2, vrh_rmse, vrh_mae, vrh_r2)
fig.show()

1. **Root Mean Square Error (RMSE):** This measures the average magnitude of the errors between the predicted values from the model and the actual values. It squares the errors before averaging to give more weight to larger errors. The RMSE for the training data is quite low at around 73.73, which suggests that the model fits the training data well. However, the RMSE for the validation data is much higher at approximately 226.88, indicating that the model doesn’t perform as well on the validation set.

2. **Mean Absolute Error (MAE):** This metric calculates the average absolute difference between the predicted values and the actual values. Unlike RMSE, MAE doesn’t square the errors, so it doesn’t penalize larger errors as heavily. The MAE is lower than the RMSE in both training (approximately 54.76) and validation (approximately 154.55), which is typically the case. However, the jump in error from training to validation suggests that the model might not generalize well to unseen data.

3. **R-squared (R²):** This is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination. The R² for both training and validation is quite high (close to 1), with training at about 0.995 and validation at about 0.948. This suggests that the model explains a large proportion of the variance in the outcome variable for both training and validation datasets.

However, the significantly higher RMSE and MAE for the validation data compared to the training data indicate that the model might be overfitting to the training data. Overfitting occurs when a model learns the training data too well, including the noise and outliers, which then harms its performance on new, unseen data.

In summary, model seems to be performing well on the training data but not as well on the validation data. This discrepancy suggests that the model may not generalize well to new data, likely due to overfitting. 

In [ ]:
plt.figure(figsize=(21, 7))

# Plot for the training set
ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted(ax1, y_train, y_train_pred, 'Training Set - True vs Predicted')

# Plot for the validation set
ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted(ax2, y_val, y_val_pred, 'Validation Set - True vs Predicted')

# Plot for the test set
ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted(ax3, y_test, y_test_pred_vp, 'Test Set - True vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
plot_results('P-Wave Velocity [m/s]', test_df_vp, y_test_pred_vp, y_test, 'VOIGT_VP', 'REUSS_VP','VRH_VP', 0, 7500, 2200, 1550)

In [ ]:
def plot_true_vs_predicted_v1(ax, true_data, predicted_data, label, marker, color):
    ax.scatter(true_data, predicted_data, alpha=0.7, label=label, marker=marker, color=color)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    ax.legend()

plt.figure(figsize=(10, 10))
ax = plt.subplot(1, 1, 1)

plot_true_vs_predicted_v1(ax, y_test, y_test_pred_vp, 'Test Set', 'o', 'red')
plot_true_vs_predicted_v1(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue')
plot_true_vs_predicted_v1(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow')

plt.tight_layout()
plt.show()

In [ ]:
def plot_true_vs_predicted_v2(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.5, label=f'{label}', color=color, marker=marker)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    # Set text for metrics below the plot area
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', transform=ax.transAxes, verticalalignment='top', horizontalalignment='left', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)

# Adjust x_pos and y_pos for each dataset to display metrics in one horizontal line below the graph
plot_true_vs_predicted_v2(ax, y_test, y_test_pred_vp, 'Testing Set', 'o', 'red', 0.66, -0.17)
plot_true_vs_predicted_v2(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue', 0.4, -0.17)
plot_true_vs_predicted_v2(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow', 0.15, -0.17)

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score

def plot_true_vs_predicted_v2(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.5, label=f'{label}', color=color, marker=marker)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    # Set text for metrics on the right of the plot area
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', transform=ax.transAxes, verticalalignment='center', horizontalalignment='left', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

plt.figure(figsize=(10, 6))  # Adjust the figure size to ensure there is enough space for the text
ax = plt.subplot(1, 1, 1)

# Fixed x_pos beyond 1 for right-side display and adjusted y_pos for vertical alignment
plot_true_vs_predicted_v2(ax, y_test, y_test_pred_vp, 'Testing Set', 'o', 'red', 1.1, 0.30)
plot_true_vs_predicted_v2(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue', 1.1, 0.45)
plot_true_vs_predicted_v2(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow', 1.1, 0.6)

plt.tight_layout(rect=[0, 0, 0.8, 1])  # Adjust the layout to prevent clipping of text
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Metrics
metrics = ['RMSE', 'MAE']
train_metrics = [train_rmse, train_mae]
val_metrics = [val_rmse, val_mae]
test_metrics = [test_rmse, test_mae]
vrh_metrics = [vrh_rmse, vrh_mae]  # Example: Replace these with actual values if they exist

n_groups = 2
index = np.arange(n_groups)
bar_width = 0.2

fig, ax = plt.subplots()

train_bar = ax.bar(index, train_metrics, bar_width, label='Train')
val_bar = ax.bar(index + bar_width, val_metrics, bar_width, label='Validation')
test_bar = ax.bar(index + 2 * bar_width, test_metrics, bar_width, label='Test')
vrh_bar = ax.bar(index + 3 * bar_width, vrh_metrics, bar_width, label='VRH')  # Adjust label as needed


ax.set_xlabel('Metrics')
ax.set_ylabel('Values')
ax.set_title('Error Metrics by Dataset')
ax.set_xticks(index + 1.5 * bar_width)
ax.set_xticklabels(metrics)
ax.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14, 5))

ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted_v1(ax1, y_train, y_train_pred, 'Training Set', 'o', 'green')

ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted_v1(ax2, y_val, y_val_pred, 'Validation Set', 's', 'blue')

ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted_v1(ax3, y_test, y_test_pred_vp, 'Test Set', 'x', 'red')

plt.tight_layout()
plt.show()


### 2. Training for Vs

In [ ]:
train_df_vs.head()

In [ ]:
target_column = 'VS'
predictor_cols = ['BOUND_WATER','CALCITE','CHLORITE','DOLOMITE',#'HEMATITE',
                    'ILLITE','K-FELDSPAR','KAOLINITE','QUARTZ','UWATER']

X = train_df_vs[predictor_cols]
y = train_df_vs[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [50, 100, 200],
    'subsample': [0.7, 1],
    'colsample_bytree': [0.7, 1],
    'reg_alpha': [ 0.1]
}

xgb_model = XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(estimator=xgb_model, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           verbose=1)
grid_search.fit(X_train_scaled, y_train)

print(f"Best parameters found: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

In [ ]:
filename = 'Model weights/XGBoost for S-Wave Velocity/best1_xgb_model.joblib'
joblib.dump(best_model, filename)

In [ ]:
filename = 'Model weights/XGBoost for S-Wave Velocity/best1_xgb_model.joblib'
best_model = joblib.load(filename)

In [ ]:
# TEST SET
X_test = test_df_vs.drop(['VS', 'RHOZ', 'VOIGT_VS', 'REUSS_VS', 'VRH_VS', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df_vs['VS']

X_test_scaled = scaler.transform(X_test)

In [ ]:
y_val_pred = best_model.predict(X_val_scaled)
y_train_pred = best_model.predict(X_train_scaled)

In [ ]:
y_test_pred_vs = best_model.predict(X_test_scaled)

In [ ]:
plt.figure(figsize=(21, 7))

# Plot for the training set
ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted(ax1, y_train, y_train_pred, 'Training Set - True vs Predicted')

# Plot for the validation set
ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted(ax2, y_val, y_val_pred, 'Validation Set - True vs Predicted')

# Plot for the test set
ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted(ax3, y_test, y_test_pred_vs, 'Test Set - True vs Predicted')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate error metrics for the training set
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate error metrics for the validation set
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Calculate error metrics for the test set
test_rmse = mean_squared_error(y_test, y_test_pred_vs, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_vs)
test_r2 = r2_score(y_test, y_test_pred_vs)

In [ ]:
plot_results('S-Wave Velocity [m/s]', test_df_vs, y_test_pred_vs, y_test, 'VOIGT_VS', 'REUSS_VS','VRH_VS', 0, 5000, 2200, 1550)

In [ ]:
plt.figure(figsize=(14, 5))

ax1 = plt.subplot(1, 3, 1)
plot_true_vs_predicted_v1(ax1, y_train, y_train_pred, 'Training Set', 'o', 'green')

ax2 = plt.subplot(1, 3, 2)
plot_true_vs_predicted_v1(ax2, y_val, y_val_pred, 'Validation Set', 's', 'blue')

ax3 = plt.subplot(1, 3, 3)
plot_true_vs_predicted_v1(ax3, y_test, y_test_pred_vs, 'Test Set', 'x', 'red')

plt.tight_layout()
plt.show()

In [ ]:
def plot_true_vs_predicted_v1(ax, true_data, predicted_data, label, marker, color):
    ax.scatter(true_data, predicted_data, alpha=0.7, label=label, marker=marker, color=color)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    ax.legend()

plt.figure(figsize=(10, 10))
ax = plt.subplot(1, 1, 1)

plot_true_vs_predicted_v1(ax, y_test, y_test_pred_vs, 'Test Set', 'o', 'red')
plot_true_vs_predicted_v1(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue')
plot_true_vs_predicted_v1(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow')

plt.tight_layout()
plt.show()

In [ ]:
def plot_true_vs_predicted_v2(ax, true_data, predicted_data, label, marker, color, x_pos, y_pos):
    mae = mean_absolute_error(true_data, predicted_data)
    r2 = r2_score(true_data, predicted_data)
    ax.scatter(true_data, predicted_data, alpha=0.5, label=f'{label}', color=color, marker=marker)
    ax.plot([true_data.min(), true_data.max()], [true_data.min(), true_data.max()], 'k--', alpha=0.5)
    ax.set_xlabel('True Values', fontweight='bold')
    ax.set_ylabel('Predicted Values', fontweight='bold')
    ax.set_title('True vs Predicted', fontweight='bold')
    # Set text for metrics below the plot area
    ax.text(x_pos, y_pos, f'{label} Metrics:\nMAE: {mae:.2f}\nR²: {r2:.2f}', transform=ax.transAxes, verticalalignment='top', horizontalalignment='left', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor='white'))

plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)

# Adjust x_pos and y_pos for each dataset to display metrics in one horizontal line below the graph
plot_true_vs_predicted_v2(ax, y_test, y_test_pred_vs, 'Testing Set', 'o', 'red', 0.66, -0.17)
plot_true_vs_predicted_v2(ax, y_val, y_val_pred, 'Validation Set', 's', 'blue', 0.4, -0.17)
plot_true_vs_predicted_v2(ax, y_train, y_train_pred, 'Training Set', '^', 'yellow', 0.15, -0.17)

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

## Calculating K and G from predicted Vp and Vs

In [ ]:
test_df_vp.head()

In [ ]:
test_df_vp.head()

In [ ]:
len(y_test_pred_vp)

In [ ]:
data = {
    'DEPT': test_df_vp['DEPT'],
    'y_test_pred_vs': y_test_pred_vs,
    'y_test_pred_vp': y_test_pred_vp,
    'RHOZ': test_df_vp['RHOZ'],
    'Voigt_Bulk': test_df_vp['Voigt_Bulk'],
    'Reuss_Bulk': test_df_vp['Reuss_Bulk'],
    'Voigt_Shear': test_df_vp['Voigt_Shear'],
    'Reuss_Shear': test_df_vp['Reuss_Shear'],
    'K_VRH': test_df_vp['K_VRH'],
    'G_VRH': test_df_vp['G_VRH'],
    'VP': test_df_vp['VP'],
    'VS': test_df_vs['VS'],
}

df_kg = pd.DataFrame(data)

df_kg['G_pred'] = ((df_kg['RHOZ']*1000) * df_kg['y_test_pred_vs']**2)/ 1e9
df_kg['K_pred'] = ((df_kg['RHOZ']*1000) * (df_kg['y_test_pred_vp']**2 - (4/3) * df_kg['y_test_pred_vs']**2))/ 1e9

df_kg['G'] = ((df_kg['RHOZ']*1000) * df_kg['VS']**2)/ 1e9
df_kg['K'] = ((df_kg['RHOZ']*1000) * (df_kg['VP']**2 - (4/3) * df_kg['VS']**2))/ 1e9

In [ ]:
df_kg.head()

In [ ]:
plot_results('Bulk Modulus [GPa]', df_kg, df_kg['K_pred'], df_kg['K'], 'Voigt_Bulk', 'Reuss_Bulk','K_VRH', 0, 75, 2200, 1660, 'yellow')

In [ ]:
plot_results('Shear Modulus [GPa]', df_kg, df_kg['G_pred'], df_kg['G'], 'Voigt_Shear', 'Reuss_Shear','G_VRH', 0, 60, 2200, 1660, 'yellow')

In [ ]:
# Checking our Results
df.head()

In [ ]:
print(test_df_vp[test_df_vp['DEPT']>1800].iloc[0])

## Double Checking our model on VW2 Data

### 1. V-Wave Velocity

In [ ]:
filename = 'Model weights/XGBoost for P-Wave Velocity/best_xgb_model.joblib'
best_model = joblib.load(filename)

In [ ]:
X_test = test_df2_vp.drop(['VP', 'RHOZ', 'VOIGT_VP', 'REUSS_VP', 'VRH_VP', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df2_vp['VP']
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_test.columns

In [ ]:
# PREDICTIONS
y_test_pred_vp = best_model.predict(X_test_scaled)

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(y_test, y_test_pred_vp, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.title('Test Set - True vs Predicted')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.show()

In [ ]:
plot_results('P-Wave Velocity [m/s]', test_df2_vp, y_test_pred_vp, y_test, 'VOIGT_VP', 'REUSS_VP','VRH_VP', 0, 7000, 2190, 1615)

### 2. S-Wave Velocity

In [ ]:
filename = 'Model weights/XGBoost for S-Wave Velocity/best1_xgb_model.joblib'
best_model = joblib.load(filename)

In [ ]:
X_test = test_df2_vs.drop(['VS', 'RHOZ', 'VOIGT_VS', 'REUSS_VS', 'VRH_VS', 'DEPT', 
                          'Voigt_Shear', 'Voigt_Bulk', 'Reuss_Shear', 'Reuss_Bulk', 
                          'K_VRH', 'G_VRH'], axis=1)
y_test = test_df2_vs['VS']

X_test_scaled = scaler.transform(X_test)

In [ ]:
y_test_pred_vs = best_model.predict(X_test_scaled)

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(y_test, y_test_pred_vs, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.title('Test Set - True vs Predicted')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.show()

In [ ]:
plot_results('S-Wave Velocity [m/s]', test_df2_vs, y_test_pred_vs, y_test, 'VOIGT_VS', 'REUSS_VS','VRH_VS', 0, 7000, 2190, 1615)

In [ ]:
data = {
    'DEPT': test_df2_vp['DEPT'],
    'y_test_pred_vs': y_test_pred_vs,
    'y_test_pred_vp': y_test_pred_vp,
    'RHOZ': test_df2_vp['RHOZ'],
    'Voigt_Bulk': test_df2_vp['Voigt_Bulk'],
    'Reuss_Bulk': test_df2_vp['Reuss_Bulk'],
    'Voigt_Shear': test_df2_vp['Voigt_Shear'],
    'Reuss_Shear': test_df2_vp['Reuss_Shear'],
    'K_VRH': test_df2_vp['K_VRH'],
    'G_VRH': test_df2_vp['G_VRH'],
    'VP': test_df2_vp['VP'],
    'VS': test_df2_vs['VS'],
}

df_kg = pd.DataFrame(data)

df_kg['G_pred'] = ((df_kg['RHOZ']*1000) * df_kg['y_test_pred_vs']**2)/ 1e9
df_kg['K_pred'] = ((df_kg['RHOZ']*1000) * (df_kg['y_test_pred_vp']**2 - (4/3) * df_kg['y_test_pred_vs']**2))/ 1e9

df_kg['G'] = ((df_kg['RHOZ']*1000) * df_kg['VS']**2)/ 1e9
df_kg['K'] = ((df_kg['RHOZ']*1000) * (df_kg['VP']**2 - (4/3) * df_kg['VS']**2))/ 1e9

In [ ]:
plot_results('Bulk Modulus [GPa]', df_kg, df_kg['K_pred'], df_kg['K'], 'Voigt_Bulk', 'Reuss_Bulk','K_VRH', 0, 100, 2190, 1615, 'yellow')

In [ ]:
plot_results('Shear Modulus [GPa]', df_kg, df_kg['G_pred'], df_kg['G'], 'Voigt_Shear', 'Reuss_Shear','G_VRH', 0, 60, 2190, 1615, 'yellow')